In [33]:
from pgmpy.models import BayesianModel
from pgmpy.factors.discrete import TabularCPD

In [34]:
PER_UNIT_GDP = 'GDP' # 'per unit GDP'
ROAD_DENSITY = 'RoD' # 'road density'
POPULATION_DENSITY = 'PD' # 'population density'
ELEVATION = 'E' # 'elevation'
SLOPE = 'S' # 'slope'
RAINFALL_DURATION = 'RaD' # 'rainfall duration'
RIVER_DENSITY = 'RiD' # 'river density'
RAINFALL_AMOUNT = 'RaA' # 'rainfall amount'
FLOOD = 'F' # 'flood'

In [35]:
values_dictionary = {
    PER_UNIT_GDP: [
        [0.1], 
        [0.2], 
        [0.7]
    ],
    ROAD_DENSITY: [
        [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
        [0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2],
        [0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7]
    ],
    POPULATION_DENSITY: [
        [0.1, 0.1, 0.1],
        [0.2, 0.2, 0.2],
        [0.7, 0.7, 0.7],
    ],
    ELEVATION: [
        [0.1],
        [0.2], 
        [0.7]
    ],
    SLOPE: [
        [0.6], 
        [0.4]
    ],
    RAINFALL_DURATION: [
        [0.1], 
        [0.2], 
        [0.7]
    ],
    RIVER_DENSITY: [
        [0.3], 
        [0.7]
    ],
    RAINFALL_AMOUNT: [
        [0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
        [0.7, 0.7, 0.7, 0.7, 0.7, 0.7],
        [0.2, 0.2, 0.2, 0.2, 0.2, 0.2],
    ],
    FLOOD: [
        [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1, 0.1],
        [0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2],
        [0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7]
    ],
}

In [36]:
table_dictionary = {
    PER_UNIT_GDP: ['High', 'Medium', 'Low'],
    ROAD_DENSITY: ['High', 'Medium', 'Low'],
    POPULATION_DENSITY: ['High', 'Medium', 'Low'],
    ELEVATION: ['High', 'Medium', 'Low'],
    SLOPE: ['Steep', 'Flat'],
    RAINFALL_DURATION: ['Long', 'Medium', 'Short'],
    RIVER_DENSITY: ['Dense', 'Sparse'],
    RAINFALL_AMOUNT: ['Huge', 'Medium', 'Little'],
    FLOOD: ['Yes', 'No']
}

In [37]:
edges = [
    (PER_UNIT_GDP, ROAD_DENSITY), 
    (PER_UNIT_GDP, POPULATION_DENSITY), 
    (POPULATION_DENSITY, ROAD_DENSITY),
    (ROAD_DENSITY, FLOOD),
    (ELEVATION, SLOPE),
    (SLOPE, FLOOD),
    (RAINFALL_DURATION, RAINFALL_AMOUNT),
    (RIVER_DENSITY, RAINFALL_AMOUNT),
    (RAINFALL_AMOUNT, FLOOD)
]

In [38]:
# Defining the model structure. We can define the network by just passing a list of edges.
model = BayesianModel(edges)

In [39]:
def pick_dictionary_subset(dictionary, keys):
    return dict((k, dictionary[k]) for k in keys if k in dictionary)

In [40]:
cpd_per_unit_gdp = TabularCPD(
    variable = PER_UNIT_GDP, 
    variable_card = len(table_dictionary[PER_UNIT_GDP]), 
    values = values_dictionary[PER_UNIT_GDP],
    state_names= pick_dictionary_subset(table_dictionary, [PER_UNIT_GDP])
)


cpd_population_density = TabularCPD(
    variable = POPULATION_DENSITY, 
    variable_card = len(table_dictionary[POPULATION_DENSITY]), 
    values = values_dictionary[POPULATION_DENSITY],
    evidence=[PER_UNIT_GDP],
    evidence_card=[len(table_dictionary[PER_UNIT_GDP])],
    state_names= pick_dictionary_subset(
        table_dictionary, [POPULATION_DENSITY, PER_UNIT_GDP]
    )
)

cpd_road_density = TabularCPD(
    variable = ROAD_DENSITY, 
    variable_card = len(table_dictionary[ROAD_DENSITY]), 
    values = values_dictionary[ROAD_DENSITY],
    evidence=[PER_UNIT_GDP, POPULATION_DENSITY],
    evidence_card=[len(table_dictionary[PER_UNIT_GDP]), len(table_dictionary[POPULATION_DENSITY])],
    state_names= pick_dictionary_subset(
        table_dictionary, [ROAD_DENSITY, PER_UNIT_GDP, POPULATION_DENSITY,]
    )
)

In [41]:
print(cpd_road_density)

+-------------+-----------+------------+-----------+-------------+-------------+-------------+----------+------------+----------+
| GDP         | GDP(High) | GDP(High)  | GDP(High) | GDP(Medium) | GDP(Medium) | GDP(Medium) | GDP(Low) | GDP(Low)   | GDP(Low) |
+-------------+-----------+------------+-----------+-------------+-------------+-------------+----------+------------+----------+
| PD          | PD(High)  | PD(Medium) | PD(Low)   | PD(High)    | PD(Medium)  | PD(Low)     | PD(High) | PD(Medium) | PD(Low)  |
+-------------+-----------+------------+-----------+-------------+-------------+-------------+----------+------------+----------+
| RoD(High)   | 0.1       | 0.1        | 0.1       | 0.1         | 0.1         | 0.1         | 0.1      | 0.1        | 0.1      |
+-------------+-----------+------------+-----------+-------------+-------------+-------------+----------+------------+----------+
| RoD(Medium) | 0.2       | 0.2        | 0.2       | 0.2         | 0.2         | 0.2      